In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
import gensim
import matplotlib.pyplot as plt
from utils import *
import logging
import json
from gensim.corpora.dictionary import Dictionary
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn import linear_model
import scipy.sparse as sps
from sklearn.model_selection import KFold
from gensim.models.tfidfmodel import TfidfModel
from itertools import chain

In [33]:
from itertools import count
import nltk
from sklearn import linear_model
from sklearn import model_selection
from nltk.corpus import stopwords
import gc
from itertools import islice

In [4]:
logging.basicConfig(level=logging.INFO)

In [4]:
fname = "data/train.csv"
data = pd.read_csv(fname, header=0)
head = data.head().copy()

In [5]:
data.question1 = data.question1.apply(normalize)
data.question2 = data.question2.apply(normalize)

INFO:root:100000
INFO:root:200000
INFO:root:300000
INFO:root:400000
INFO:root:500000
INFO:root:600000
INFO:root:700000
INFO:root:800000


In [6]:
data.to_pickle("cdata/train_cleaned.pickle")

In [6]:
data = pd.read_pickle("cdata/train_cleaned.pickle")

In [111]:
stemed_data = data

In [115]:
progress = 0
def stem_list(x):
    global progress
    progress+=1
    if not progress%100000:
        print (progress)
    return list(map(stemer.stem, x))

In [116]:
stemed_data.question1 = stemed_data.question1.apply(stem_list)
stemed_data.question2 = stemed_data.question2.apply(stem_list)


20000
40000
60000
80000
100000
120000
140000
160000
180000
200000
220000
240000
260000
280000
300000
320000
340000
360000
380000
400000
420000
440000
460000
480000
500000
520000
540000
560000
580000
600000
620000
640000
660000
680000
700000
720000
740000
760000
780000
800000


In [117]:
stemed_data.to_pickle("cdata/train_cleaned_stemed.pickle")

In [4]:
data = pd.read_pickle("cdata/train_cleaned.pickle")

In [5]:
stop = set(stopwords.words('english'))
def kill_stop(x):
    return [y for y in x if y not in stop]

nostop_data = data
nostop_data.question1 = nostop_data.question1.apply(kill_stop)
nostop_data.question2 = nostop_data.question2.apply(kill_stop)

In [6]:
data.to_pickle("cdata/train_cleaned_stemed_nostop.pickle")

In [5]:
data = pd.read_pickle("cdata/train_cleaned_stemed_nostop.pickle")

In [7]:
data.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,"[step, step, guide, invest, share, market, india]","[step, step, guide, invest, share, market]",0
1,1,3,4,"[story, kohinoor, (, koh, noor, ), diamond]","[would, happen, indian, government, stole, koh...",0
2,2,5,6,"[increase, speed, internet, connection, using,...","[internet, speed, increased, hacking, dns]",0
3,3,7,8,"[mentally, lonely, solve]","[find, remainder, [, math, ], 23, ^, {, quoted...",0
4,4,9,10,"[one, dissolve, water, quickly, sugar, salt, m...","[fish, would, survive, salt, water]",0


In [8]:
Y = data.is_duplicate.values

In [9]:
all_tokens = (list(data.question1.values) + list(data.question2.values))
vectorizer = Dictionary(all_tokens)

INFO:gensim.corpora.dictionary:adding document #0 to Dictionary(0 unique tokens: [])
INFO:gensim.corpora.dictionary:adding document #10000 to Dictionary(11626 unique tokens: ['msm8916', 'army', 'analogue', 'drugs', 'voltmeter']...)
INFO:gensim.corpora.dictionary:adding document #20000 to Dictionary(16700 unique tokens: ['msm8916', 'accomplished', 'army', 'waters', 'analogue']...)
INFO:gensim.corpora.dictionary:adding document #30000 to Dictionary(20468 unique tokens: ['msm8916', 'accomplished', 'army', 'waters', 'analogue']...)
INFO:gensim.corpora.dictionary:adding document #40000 to Dictionary(23509 unique tokens: ['msm8916', '999', 'clindamycin', '2400kms', 'truckers']...)
INFO:gensim.corpora.dictionary:adding document #50000 to Dictionary(26383 unique tokens: ['msm8916', '999', 'timesheets', 'clindamycin', '2400kms']...)
INFO:gensim.corpora.dictionary:adding document #60000 to Dictionary(28779 unique tokens: ['msm8916', '999', 'timesheets', 'clindamycin', '2400kms']...)
INFO:gensim.

In [10]:
vectorizer.filter_extremes(no_below=5, no_above=0.1)
vocab_size = len(vectorizer.dfs)

INFO:gensim.corpora.dictionary:discarding 56727 tokens: [('tiago', 4), ('motorolla', 1), ('dcx3400', 2), ('biprism', 1), ('slits', 4), ('fresnel', 4), ('aoa', 2), ('rexnord', 1), ('israil', 1), ('sincerity', 1)]...
INFO:gensim.corpora.dictionary:keeping 29964 tokens which were in no less than 5 and no more than 80858 (=10.0%) documents
INFO:gensim.corpora.dictionary:resulting dictionary: Dictionary(29964 unique tokens: ['xiaoping', 'antifreeze', 'graviton', 'tomcat', 'purge']...)


In [16]:
questions = list(map(vectorizer.doc2bow, list(data.question1.values) + list(data.question2.values)))


In [165]:
lda = gensim.models.ldamulticore.LdaMulticore(questions, id2word=vectorizer, num_topics=100, workers=2, passes=1)

INFO:gensim.models.ldamodel:using symmetric alpha at 0.01
INFO:gensim.models.ldamodel:using symmetric eta at 3.324026060364313e-05
INFO:gensim.models.ldamodel:using serial LDA version on this node
INFO:gensim.models.ldamulticore:running online LDA training, 100 topics, 1 passes over the supplied corpus of 808580 documents, updating every 4000 documents, evaluating every ~40000 documents, iterating 50x with a convergence threshold of 0.001000
INFO:gensim.models.ldamulticore:training LDA model using 2 processes
INFO:gensim.models.ldamulticore:PROGRESS: pass 0, dispatched chunk #0 = documents up to #2000/808580, outstanding queue size 1
INFO:gensim.models.ldamulticore:PROGRESS: pass 0, dispatched chunk #1 = documents up to #4000/808580, outstanding queue size 2
INFO:gensim.models.ldamulticore:PROGRESS: pass 0, dispatched chunk #2 = documents up to #6000/808580, outstanding queue size 3
INFO:gensim.models.ldamulticore:PROGRESS: pass 0, dispatched chunk #3 = documents up to #8000/808580, ou

In [204]:
lda.save("models/lda_question.pickle")

INFO:gensim.utils:saving LdaState object under models/lda_question.pickle.state, separately None
INFO:gensim.utils:saved models/lda_question.pickle.state
INFO:gensim.utils:saving LdaMulticore object under models/lda_question.pickle, separately ['expElogbeta', 'sstats']
INFO:gensim.utils:not storing attribute state
INFO:gensim.utils:not storing attribute dispatcher
INFO:gensim.utils:not storing attribute id2word
INFO:gensim.utils:storing np array 'expElogbeta' to models/lda_question.pickle.expElogbeta.npy
INFO:gensim.utils:saved models/lda_question.pickle


In [11]:
lda = gensim.models.ldamulticore.LdaMulticore.load("models/lda_question.pickle")

INFO:gensim.utils:loading LdaMulticore object from models/lda_question.pickle
INFO:gensim.utils:loading expElogbeta from models/lda_question.pickle.expElogbeta.npy with mmap=None
INFO:gensim.utils:setting ignored attribute state to None
INFO:gensim.utils:setting ignored attribute dispatcher to None
INFO:gensim.utils:setting ignored attribute id2word to None
INFO:gensim.utils:loaded models/lda_question.pickle
INFO:gensim.utils:loading LdaMulticore object from models/lda_question.pickle.state
INFO:gensim.utils:loaded models/lda_question.pickle.state


In [12]:
def lda_pipe(x):
    sparse = vectorizer.doc2bow(x)
    lda_sparse = lda[sparse]
    return lda_sparse

In [25]:
from multiprocessing import Pool

def pipe(pom):
    i = pom[0]
    q1 = pom[1][0]
    q2 = pom[1][1]
    q1 = lda_pipe(q1)
    q2 = lda_pipe(q2)
    if not i%100:
        print(i)
    return [q1, q2]

pool    = Pool(processes=8)

results = pool.map(pipe, enumerate(zip(data.question1, data.question2)))
#results = pool.map(pipe, enumerate(islice(zip(data.question1, data.question2), 1000)))


0
25300
63200
12700
88500
50600
38000
100
75900
25400
63300
12800
88600
50700
38100
200
76000
25500
63400
12900
88700
50800
38200
300
76100
25600
63500
13000
88800
50900
38300
400
76200
25700
63600
13100
88900
51000
38400
500
76300
25800
63700
13200
89000
51100
38500
600
76400
25900
63800
13300
89100
51200
700
38600
76500
26000
63900
13400
89200
51300
800
38700
76600
26100
64000
13500
89300
51400
900
38800
76700
64100
26200
13600
89400
51500
1000
76800
38900
64200
26300
13700
89500
51600
76900
1100
39000
64300
26400
13800
89600
51700
77000
1200
39100
64400
26500
13900
89700
51800
77100
1300
39200
64500
26600
14000
89800
51900
77200
1400
39300
64600
26700
14100
89900
52000
77300
1500
39400
64700
26800
14200
90000
52100
77400
1600
39500
64800
26900
14300
90100
52200
77500
1700
39600
64900
27000
14400
90200
52300
77600
1800
39700
65000
27100
14500
90300
52400
77700
1900
65100
39800
27200
14600
90400
52500
77800
2000
65200
39900
27300
14700
90500
52600
77900
2100
65300
40000
27400
14800
90

In [28]:
import pickle

In [30]:
pickle.dump(results, open("data/lda_100_embeded.pickle", "wb"))

In [14]:
from itertools import islice

In [15]:
sample = islice(zip(data.question1, data.question2),10)

In [49]:
lda_X = sps.lil_matrix((Y.size, 100))
for i, (q1, q2) in enumerate(results):
    qq1 = dict(q1)
    qq2 = dict(q2)
    nq1 = sum(qq1.values())
    nq2 = sum(qq2.values())
    indices = []
    values = []
    for sk in set(qq1.keys())&set(qq2.keys()):
        indices.append(sk)
        values.append((qq1[sk]*qq2[sk])/(nq1+nq2))
    lda_X[i, [indices]] = values
    
    if not i%10000:
        print(i)
        
lda_X = lda_X.tocsr()

0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000


In [56]:
from sklearn.neural_network import MLPClassifier

In [ ]:
def scoring(cls, X, Y):
    return loss(Y, clf.predict_proba(X[test_index])[:,1])
param_grid = {
    "max_depth": [2, 1],
    "learning_rate": [0.1, 1],
    "n_estimators": [100],
    "reg_alpha": [0.1],
    "reg_lambda": [0.1]
}
logistic_grid = {
    "penalty": ["l1","l2"],
    "C": [0.1, 1]
}
mlp_grid = {
    "hidden_layer_sizes": [10, 100],
    "alpha": [0.0001, 0.001, 0.01],
    "activation": ['identity', 'logistic', 'tanh', 'relu']
}
#cls = XGBClassifier()
#grid = model_selection.GridSearchCV(cls, param_grid, verbose=10, n_jobs=2, cv=2)

#cls = linear_model.LogisticRegression(class_weight=None, max_iter=100, verbose=100, warm_start=False, n_jobs=4)
#grid = model_selection.GridSearchCV(cls, logistic_grid, verbose=10, n_jobs=2, cv=2)
#cls = linear_model.RidgeClassifier()
#cll = linear_model.PassiveAggressiveClassifier(verbose=100, n_iter=10)

cls = MLPClassifier(max_iter=100, verbose=100, warm_start=False)
grid = model_selection.GridSearchCV(cls, mlp_grid, verbose=10, n_jobs=2, cv=2)


grid.fit(lda_X, Y)
    

Fitting 2 folds for each of 24 candidates, totalling 48 fits
[CV] activation=identity, alpha=0.0001, hidden_layer_sizes=10 ........
[CV] activation=identity, alpha=0.0001, hidden_layer_sizes=10 ........
Iteration 1, loss = 0.64854507
Iteration 1, loss = 0.64882224
Iteration 2, loss = 0.61692151
Iteration 2, loss = 0.61726957
Iteration 3, loss = 0.59997032
Iteration 3, loss = 0.60030712
Iteration 4, loss = 0.59591274
Iteration 4, loss = 0.59629194
Iteration 5, loss = 0.59515977
Iteration 5, loss = 0.59556380
Iteration 6, loss = 0.59499065
Iteration 6, loss = 0.59536959
Iteration 7, loss = 0.59485455
Iteration 7, loss = 0.59533080
Iteration 8, loss = 0.59488946
Iteration 8, loss = 0.59530432
Iteration 9, loss = 0.59489618
Iteration 9, loss = 0.59532859
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
[CV]  activation=identity, alpha=0.0001, hidden_layer_sizes=10, score=0.666747, total=   4.4s
[CV] activation=identity, alpha=0.0001, hidden_layer_s

[Parallel(n_jobs=2)]: Done   1 tasks      | elapsed:    4.5s


Iteration 10, loss = 0.59482590
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
[CV]  activation=identity, alpha=0.0001, hidden_layer_sizes=10, score=0.665415, total=   5.0s
[CV] activation=identity, alpha=0.0001, hidden_layer_sizes=100 .......
Iteration 1, loss = 0.62855424
Iteration 1, loss = 0.63456191
Iteration 2, loss = 0.59671354
Iteration 2, loss = 0.59801816
Iteration 3, loss = 0.59558070
Iteration 3, loss = 0.59603902
Iteration 4, loss = 0.59554789
Iteration 4, loss = 0.59589978
Iteration 5, loss = 0.59557162
Iteration 5, loss = 0.59592214
Iteration 6, loss = 0.59548869
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
[CV]  activation=identity, alpha=0.0001, hidden_layer_sizes=100, score=0.668354, total=  11.9s
[CV] activation=identity, alpha=0.001, hidden_layer_sizes=10 .........
Iteration 6, loss = 0.59591968
Iteration 1, loss = 0.65190756
Iteration 7, loss = 0.59593432
Training loss did not

[Parallel(n_jobs=2)]: Done   4 tasks      | elapsed:   18.5s


Iteration 3, loss = 0.60320381
Iteration 1, loss = 0.66280704
Iteration 4, loss = 0.59818626
Iteration 2, loss = 0.63774853
Iteration 5, loss = 0.59713714
Iteration 3, loss = 0.61313920
Iteration 6, loss = 0.59687450
Iteration 4, loss = 0.60161650
Iteration 7, loss = 0.59668818
Iteration 5, loss = 0.59861006
Iteration 8, loss = 0.59672917
Iteration 6, loss = 0.59796351
Iteration 9, loss = 0.59666672
Iteration 7, loss = 0.59769186
Iteration 10, loss = 0.59659455
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
[CV]  activation=identity, alpha=0.001, hidden_layer_sizes=10, score=0.667681, total=   5.4s
[CV] activation=identity, alpha=0.001, hidden_layer_sizes=100 ........
Iteration 8, loss = 0.59757305
Iteration 9, loss = 0.59752395
Iteration 1, loss = 0.63186166
Iteration 10, loss = 0.59743521
Iteration 11, loss = 0.59738945
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 2, loss = 0.60025569


...

Iteration 60, loss = 0.60530237
Iteration 1, loss = 0.65955490
Iteration 61, loss = 0.60529679
Iteration 2, loss = 0.65852747
Iteration 62, loss = 0.60516011
Iteration 3, loss = 0.65875239
Iteration 63, loss = 0.60527185
Iteration 4, loss = 0.65859624
Iteration 64, loss = 0.60490188
Iteration 5, loss = 0.65848044
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 65, loss = 0.60488779
[CV]  activation=logistic, alpha=0.01, hidden_layer_sizes=100, score=0.630801, total=  22.7s
[CV] activation=logistic, alpha=0.01, hidden_layer_sizes=100 .........
Iteration 66, loss = 0.60491853
Iteration 1, loss = 0.65879548
Iteration 67, loss = 0.60470384
Iteration 2, loss = 0.65868071
Iteration 68, loss = 0.60471733
Iteration 69, loss = 0.60471802
Iteration 3, loss = 0.65861034
Iteration 70, loss = 0.60481048
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 4, loss = 0.65870425
[CV]  activation=logist

[Parallel(n_jobs=2)]: Done  28 tasks      | elapsed:  9.7min


Iteration 7, loss = 0.59714305
Iteration 1, loss = 0.65244111
Iteration 8, loss = 0.59702571
Iteration 2, loss = 0.62959660
Iteration 9, loss = 0.59686584
Iteration 3, loss = 0.60834166
Iteration 10, loss = 0.59684335
Iteration 4, loss = 0.60017727
Iteration 11, loss = 0.59679826
Iteration 5, loss = 0.59809487
Iteration 12, loss = 0.59676418
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
[CV]  activation=tanh, alpha=0.001, hidden_layer_sizes=10, score=0.665000, total=   9.0s
[CV] activation=tanh, alpha=0.001, hidden_layer_sizes=100 ............
Iteration 6, loss = 0.59755869
Iteration 7, loss = 0.59734997
Iteration 8, loss = 0.59729506
Iteration 9, loss = 0.59724112
Iteration 1, loss = 0.63327614
Iteration 10, loss = 0.59714479
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
[CV]  activation=tanh, alpha=0.001, hidden_layer_sizes=10, score=0.662849, total=   6.9s
[CV] activation=tanh, alpha=0.001, hid

[Parallel(n_jobs=2)]: Done  37 tasks      | elapsed: 12.7min


Iteration 87, loss = 0.54119623
Iteration 88, loss = 0.54094785
Iteration 1, loss = 0.63976229
Iteration 89, loss = 0.54102294
Iteration 90, loss = 0.54089617
Iteration 2, loss = 0.59876494
Iteration 91, loss = 0.54089691
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
[CV]  activation=relu, alpha=0.0001, hidden_layer_sizes=10, score=0.690402, total=  56.6s
[CV] activation=relu, alpha=0.0001, hidden_layer_sizes=100 ...........
Iteration 1, loss = 0.64158567
Iteration 3, loss = 0.59096110
Iteration 2, loss = 0.59894281
Iteration 4, loss = 0.58502048
Iteration 5, loss = 0.57697221
Iteration 3, loss = 0.58608143
Iteration 6, loss = 0.56825867
Iteration 4, loss = 0.57623546
Iteration 5, loss = 0.56665055
Iteration 7, loss = 0.56040970
Iteration 6, loss = 0.55849958
Iteration 8, loss = 0.55408669
Iteration 7, loss = 0.55203958
Iteration 9, loss = 0.54966044
Iteration 8, loss = 0.54730787
Iteration 10, loss = 0.54618639
Iteration 11, loss = 0.543620

In [121]:
same = [list(set([w for w in x[3] if w in x[4]])) for x in data.values]
diff = [list(set([w for w in x[3] if w not in x[4]])) + [w for w in x[4] if w not in x[3]] for x in data.values]

In [ ]:
lda_X1 = data.question1.apply(lda).values

In [125]:
X = sps.lil_matrix((Y.size, len(vectorizer.dfs)*2))
for i, (s, d) in enumerate(zip(same, diff)):
    s_bow = vectorizer.doc2bow(s)
    d_bow = vectorizer.doc2bow(d)
    indices = [x[0] for x in s_bow] + [vocab_size + x[0] for x in d_bow]
    X[i, [indices]] = 1.
    if not i%10000:
        print(i)
    

0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000


In [38]:
from xgboost import XGBClassifier

/usr/local/lib/python3.5/dist-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


RidgeClassifier(alpha=1.0, class_weight=None, copy_X=True, fit_intercept=True,
        max_iter=None, normalize=False, random_state=None, solver='auto',
        tol=0.001)

In [46]:
cls.get_params().keys()

dict_keys(['copy_X', 'tol', 'fit_intercept', 'class_weight', 'normalize', 'max_iter', 'solver', 'alpha', 'random_state'])

In [127]:
nice_X = X.tocsr()

In [128]:
lda = LdaMulticore

<404290x60168 sparse matrix of type '<class 'numpy.float64'>'
	with 4648100 stored elements in Compressed Sparse Row format>

In [ ]:
XGBClassifier()
max_depth=3, learning_rate=0.1, n_estimators=100, silent=True, objective='binary:logistic', booster='gbtree', n_jobs=1, nthread=None, gamma=0, min_child_weight=1, max_delta_step=0, subsample=1, colsample_bytree=1, colsample_bylevel=1, reg_alpha=0, reg_lambda=1, scale_pos_weight=1, base_score=0.5, random_state=0, seed=None, missing=None, 

In [151]:
def scoring(cls, X, Y):
    return loss(Y, clf.predict_proba(X[test_index])[:,1])
param_grid = {
    "max_depth": [3],
    "learning_rate": [0.1, 1],
    "n_estimators": [100],
    "reg_alpha": [0.1],
    "reg_lambda": [0.1]
}
cls = XGBClassifier()
grid = model_selection.GridSearchCV(cls, param_grid, verbose=10, n_jobs=2, cv=2)

grid.fit(nice_X, Y)
    

Fitting 2 folds for each of 2 candidates, totalling 4 fits
[CV] learning_rate=0.1, reg_alpha=0.1, max_depth=3, n_estimators=100, reg_lambda=0.1 
[CV] learning_rate=0.1, reg_alpha=0.1, max_depth=3, n_estimators=100, reg_lambda=0.1 
[CV] learning_rate=1, reg_alpha=0.1, max_depth=3, n_estimators=100, reg_lambda=0.1 
[CV] learning_rate=1, reg_alpha=0.1, max_depth=3, n_estimators=100, reg_lambda=0.1 
[CV]  learning_rate=1, reg_alpha=0.1, max_depth=3, n_estimators=100, reg_lambda=0.1, score=0.745407, total=   8.2s
[CV]  learning_rate=1, reg_alpha=0.1, max_depth=3, n_estimators=100, reg_lambda=0.1, score=0.746903, total=   8.2s


[Parallel(n_jobs=2)]: Done   1 tasks      | elapsed:   11.1s
[Parallel(n_jobs=2)]: Done   2 out of   4 | elapsed:   11.2s remaining:   11.2s
Process ForkPoolWorker-31:
Process ForkPoolWorker-32:
Traceback (most recent call last):
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/usr/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.5/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/usr/local/lib/python3.5/dist-packages/sklearn/externals/joblib/pool.py", line 360, in get
    racquire()
KeyboardInterrupt


KeyboardInterrupt: 

In [153]:
kf = KFold(n_splits=10)


#clf = LogisticRegression(class_weight=None, max_iter=100, verbose=100, warm_start=False, n_jobs=4)
#cls = XGBClassifier()
cls = RidgeClassifier()
#clf = PassiveAggressiveClassifier(verbose=100, n_iter=10)
for train_index, test_index in kf.split(nice_X):
    clf.fit(nice_X[train_index],Y[train_index])
    #probs = clf.predict_proba(X[test_index])[:,1]
    probs = clf.predict(X[test_index])
    
    print(loss(Y[test_index], probs))
    break

-- Epoch 1
Norm: 88.63, NNZs: 48318, Bias: -0.259562, T: 363836, Avg. loss: 0.627991
Total training time: 0.09 seconds.
-- Epoch 2
Norm: 115.07, NNZs: 50383, Bias: -0.296454, T: 727672, Avg. loss: 0.597837
Total training time: 0.18 seconds.
-- Epoch 3
Norm: 134.55, NNZs: 51132, Bias: -0.338178, T: 1091508, Avg. loss: 0.581157
Total training time: 0.27 seconds.
-- Epoch 4
Norm: 150.34, NNZs: 51500, Bias: -0.403964, T: 1455344, Avg. loss: 0.569793
Total training time: 0.36 seconds.
-- Epoch 5
Norm: 163.77, NNZs: 51747, Bias: -0.382112, T: 1819180, Avg. loss: 0.561500
Total training time: 0.45 seconds.
-- Epoch 6
Norm: 175.41, NNZs: 51921, Bias: -0.380413, T: 2183016, Avg. loss: 0.554646
Total training time: 0.54 seconds.
-- Epoch 7
Norm: 186.03, NNZs: 52044, Bias: -0.368573, T: 2546852, Avg. loss: 0.549532
Total training time: 0.63 seconds.
-- Epoch 8
Norm: 195.58, NNZs: 52136, Bias: -0.509633, T: 2910688, Avg. loss: 0.545117
Total training time: 0.72 seconds.
-- Epoch 9
Norm: 204.02, NN

LogisticRegression => 0.3936

xgboost => terible 6.23, depth must be 1 or 2, 3 is overfit

PassiveAggressiveClassifier => 8

lda concat => 0.6
